In [1]:
import pandas as pd
import numpy as np
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.explainers import MetricTextExplainer
from aif360.algorithms.preprocessing import Reweighing, DisparateImpactRemover, LFR
from sklearn.preprocessing import StandardScaler


c:\Users\srinivas\anaconda3\Lib\site-packages\torch\_functorch\deprecated.py:61: UserWarning: We've integrated functorch into PyTorch. As the final step of the integration, functorch.vmap is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use torch.vmap instead; see the PyTorch 2.0 release notes and/or the torch.func migration guide for more details https://pytorch.org/docs/master/func.migrating.html
  warn_deprecated('vmap', 'torch.vmap')


In [2]:
from bias_processor import *

In [3]:
df = pd.read_csv('MEPSDataset19.csv')
bias = BiasMitigator()


In [4]:
df.head()


,Unnamed: 0,AGE,RACE,PCS42,MCS42,K6SUM42,REGION=1,REGION=2,REGION=3,REGION=4,...,EMPST=4,POVCAT=1,POVCAT=2,POVCAT=3,POVCAT=4,POVCAT=5,INSCOV=1,INSCOV=2,INSCOV=3,UTILIZATION
0,0,53.0,1.0,25.93,58.47,3.0,0.0,1.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,1,56.0,1.0,20.42,26.57,17.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
2,3,23.0,1.0,53.12,50.33,7.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,4,3.0,1.0,-1.00,-1.00,-1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,5,27.0,0.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [5]:
protected = [{'RACE': 1.0}]
unprotected = [{'RACE': 0.0}]

In [6]:
sens_attr = 'RACE'
label = 'UTILIZATION'

In [7]:
df2 = bias.compute_bias_metrics(df, unprotected, protected, sens_attr, label)
df2.head()

,Mean Difference,Consistency,Statistical Parity Difference,Disparate Impact,Mean Difference Explanation,Consistency Explanation,Statistical Parity Difference Explanation,Disparate Impact Explanation
Values,-0.130083,[0.8247378395451697],-0.130083,0.490478,Mean difference (mean label value on unprivile...,"Consistency (Zemel, et al. 2013): [0.82473784]",Statistical parity difference (probability of ...,Disparate impact (probability of favorable out...


In [9]:
# threshold=0.22
df3 = bias.preprocess(df, 'reweighing', sens_attr, unprotected, protected, label)
df3.head()

,Unnamed: 0,AGE,RACE,PCS42,MCS42,K6SUM42,REGION=1,REGION=2,REGION=3,REGION=4,...,POVCAT=1,POVCAT=2,POVCAT=3,POVCAT=4,POVCAT=5,INSCOV=1,INSCOV=2,INSCOV=3,label,weights
0,0.0,53.0,1.0,25.93,58.47,3.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.672529
1,1.0,56.0,1.0,20.42,26.57,17.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.672529
2,3.0,23.0,1.0,53.12,50.33,7.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.112267
3,4.0,3.0,1.0,-1.00,-1.00,-1.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.112267
4,5.0,27.0,0.0,-1.00,-1.00,-1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.946869


In [10]:
df4=bias.compute_bias_metrics(df3, unprotected, protected, sens_attr, label='label')

In [11]:
df4

,Mean Difference,Consistency,Statistical Parity Difference,Disparate Impact,Mean Difference Explanation,Consistency Explanation,Statistical Parity Difference Explanation,Disparate Impact Explanation
Values,-0.130083,[0.8248389134554664],-0.130083,0.490478,Mean difference (mean label value on unprivile...,"Consistency (Zemel, et al. 2013): [0.82483891]",Statistical parity difference (probability of ...,Disparate impact (probability of favorable out...
